In [1]:
import FailtogetConv_error as fe
fe.fail_to_get_conv()

1 Physical GPUs, 1 Logical GPUs


In [2]:
import os
from glob import glob
from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

In [3]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## build Model

In [4]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

## data preprocess

In [6]:
os.listdir('../dataset/cifar/')

['labels.txt', 'test', 'train']

In [7]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [8]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [9]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [10]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [11]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [12]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [15]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Callbacks

In [16]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20210128-080359'

In [17]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [18]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

In [19]:
%tensorboard --logdir logs --port 8115

## training

In [20]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(train_dataset,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=test_dataset,
                    validation_steps=validation_steps,
                    epochs=num_epochs,
                    callbacks=[tensorboard])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 1/31 [..............................] - ETA: 0s - loss: 2.3169 - accuracy: 0.1250WARNING:tensorflow:From /home/jovyan/.venv/tf2.3.0-keras2.4.0-py3.7-cuda10.1/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
31/31 [==============================] - 1s 30ms/step - loss: 2.3001 - accuracy: 0.1178 - val_loss: 2.2746 - val_accuracy: 0.1129
Epoch 2/10
31/31 [==============================] - 1s 21ms/step - loss: 2.2454 - accuracy: 0.1291 - val_loss: 2.2398 - val_accuracy: 0.1613
Epoch 3/10
31/31 [==============================] - 1s 21ms/step - loss: 2.1877 - accuracy: 0.1601 - val_loss: 2.0891 - val_accuracy: 0.2087
Epoch 4/10
31/31 [==============================] - 1s 19ms/step - loss: 2.1489 - accuracy: 0.1994 

## Lambda Callbacks
https://www.tensorflow.org/tensorboard/image_summaries#setup

## Expert

In [21]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [22]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [23]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [24]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [25]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [26]:
file_writer = tf.summary.create_file_writer(logdir)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step+(step*epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step=step+(step*epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step=step+(step*epoch))

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

## Learning Rate Scheduler

In [28]:
import math

def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001*math.exp(0.1*(10-epoch))

In [29]:
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [30]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)

Epoch 1/10
31/31 [==============================] - 0s 13ms/step - loss: 3.9571 - accuracy: 0.4442 - val_loss: 1.9584 - val_accuracy: 0.3538
Epoch 2/10
31/31 [==============================] - 0s 12ms/step - loss: 1.5288 - accuracy: 0.4990 - val_loss: 1.7868 - val_accuracy: 0.3841
Epoch 3/10
31/31 [==============================] - 0s 10ms/step - loss: 1.2533 - accuracy: 0.5692 - val_loss: 1.8944 - val_accuracy: 0.3861
Epoch 4/10
31/31 [==============================] - 0s 11ms/step - loss: 1.2759 - accuracy: 0.5897 - val_loss: 1.9213 - val_accuracy: 0.4103
Epoch 5/10
31/31 [==============================] - 0s 13ms/step - loss: 1.1681 - accuracy: 0.5971 - val_loss: 1.8779 - val_accuracy: 0.4103
Epoch 6/10
31/31 [==============================] - 0s 10ms/step - loss: 1.0517 - accuracy: 0.6483 - val_loss: 1.9377 - val_accuracy: 0.4294
Epoch 7/10
31/31 [==============================] - 0s 10ms/step - loss: 1.0452 - accuracy: 0.6498 - val_loss: 1.9447 - val_accuracy: 0.4254
Epoch 8/10
31

## Checkpoint

In [31]:
save_path = 'checkpoints'

In [34]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path,
                                                monitor='val_acc',
                                                verbose=1,
                                                save_best_only=True)

In [35]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[checkpoint]
)

Epoch 1/10
31/31 [==============================] - 4s 121ms/step - loss: 0.7636 - accuracy: 0.7386 - val_loss: 2.0739 - val_accuracy: 0.4143
Epoch 2/10
31/31 [==============================] - 0s 11ms/step - loss: 0.7742 - accuracy: 0.7200 - val_loss: 2.0110 - val_accuracy: 0.4234
Epoch 3/10
31/31 [==============================] - 0s 10ms/step - loss: 0.7536 - accuracy: 0.7448 - val_loss: 2.1231 - val_accuracy: 0.4264
Epoch 4/10
31/31 [==============================] - 0s 11ms/step - loss: 0.7322 - accuracy: 0.7593 - val_loss: 2.1140 - val_accuracy: 0.4315
Epoch 5/10
31/31 [==============================] - 0s 10ms/step - loss: 0.7469 - accuracy: 0.7583 - val_loss: 2.0072 - val_accuracy: 0.4395
Epoch 6/10
31/31 [==============================] - 0s 11ms/step - loss: 0.7033 - accuracy: 0.7541 - val_loss: 2.0002 - val_accuracy: 0.4355
Epoch 7/10
31/31 [==============================] - 0s 10ms/step - loss: 0.7610 - accuracy: 0.7440 - val_loss: 2.0355 - val_accuracy: 0.4597
Epoch 8/10
3